<a href="https://colab.research.google.com/github/phyml4e/PINNs/blob/main/COMM_PINN/umat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math

# umat functions

In [ ]:
# material model
def TSL(gv, d_i, xid_i, prop):

  ks1 = prop[0]             # Initial stiffness
  ks2 = prop[1]             # Initial stiffness
  kn  = prop[2]             # Initial stiffness
  h1  = prop[3]             # Interface damage hardening parameter 1
  h2  = prop[4]             # Interface damage hardening parameter 2
  y0  = prop[5]             # damage onset

  K   = np.array([[ks1, 0, 0], [0, ks2, 0], [0, 0, kn]])
  En  = np.dot(np.dot(gv, K), gv)
  YT = (1 - d_i)*En
  phi = YT - (y0 + h1*(1 - math.exp(-h2*xid_i)))    # Damage criteria

  if phi < 0:              # elastic step
    d   = d_i
    xid = xid_i
    #tngMM = ((1-d)**2)*k0
  else:                    # damage step
    d   = d_i
    xid = xid_i
    Y   = (1 - d)*En
    rd = np.zeros([2, 1])
    rd[0,0] = d - d_i - (xid - xid_i)
    rd[1,0] = Y - (y0 + h1*(1 - math.exp(-h2*xid)))

    while np.linalg.norm(rd) > 10**(-13):

      Kd      = np.zeros([2,2])
      Kd[0,0] = 1
      Kd[0,1] = -1
      Kd[1,0] = -En
      Kd[1,1] = -h1*h2*math.exp(-h2*xid)

      Dsol    = np.linalg.lstsq(-Kd, rd,rcond=None)[0]
      d       = d + Dsol[0,0]
      xid     = xid + Dsol[1,0]
      Y       = (1 - d)*En

      rd[0,0] = d - d_i - (xid - xid_i)                   # final evol eq
      rd[1,0] = Y - (y0 + h1*(1 - math.exp(-h2*xid)))     # final yeild fn

    if d < d_i:
      d = d_i
      xid = xid_i

  tra = ((1-d)**2)*np.dot(K, np.transpose(gv))
  ene = ((1-d)**2)*0.5*En + (h1*(xid + (math.exp(-h2*xid)-1)/h2))

  return tra,d,xid,ene

In [ ]:
# feed_forward material model
def ReLU(x):
    return x * (x > 0)
def feed_forward(inputs, weights, biases):

    layer_inputs = np.array(inputs).reshape(1, -1)
    for i in range(len(weights)-1):
        layer_weights = weights[i]
        layer_biases = biases[i]
        layer_outputs = np.dot(layer_inputs, layer_weights) + layer_biases
        layer_outputs = ReLU(layer_outputs)  # Apply the tanh activation function
        layer_inputs = layer_outputs

    last_layer_weights = weights[-1]
    last_layer_biases = biases[-1]
    last_layer_outputs = np.dot(layer_inputs, last_layer_weights) + last_layer_biases

    return last_layer_outputs

# initialization

In [ ]:
# Material Parameters
ks1  = 0.5         # Initial stiffness
ks2  = 2.0         # Initial stiffness
kn   = 5.0         # Initial stiffness
h1   = 2.0         # Interface damage hardening parameter 1
h2   = 1.0         # Interface damage hardening parameter 2
y0   = 0.1         # Damage onset
prop = [ks1, ks2, kn, h1, h2, y0]

In [ ]:
step_size = 0.05
num_steps = int(1 / step_size) + 1
Gap = np.zeros((num_steps,3))
Tra = np.zeros((num_steps,3))
Dmg = np.zeros((num_steps))
Xid = np.zeros((num_steps))
Ene = np.zeros((num_steps))

for i in range(num_steps):
    t = i * step_size
    Gap[i,:] = np.array([0.5*abs(np.sin(3.1415*2*t)), 0.5*abs(np.cos(3.1415*2*t)), 0.5*t**2])

In [ ]:
x1_NN = Gap[:,0]
x1_NN = x1_NN.reshape(-1,1)
x2_NN = Gap[:,1]
x2_NN = x2_NN.reshape(-1,1)
x3_NN = Gap[:,2]
x3_NN = x3_NN.reshape(-1,1)
x4_NN = np.zeros_like(x1_NN)
x5_NN = np.zeros_like(x1_NN)
trc_N = np.zeros_like(Gap)

In [ ]:
weights = []
with open('cz_3d.txt', 'r') as file:
    array_lines = file.read().split('---\n')
    for array_line in array_lines:
        if array_line.strip():
            lines = array_line.strip().split('\n')
            shape_line = lines[0]
            array_data = np.genfromtxt(lines[1:], delimiter=',')
            weights.append(array_data)

In [ ]:
weights1 = np.array(weights[0])
weights2 = np.array(weights[1])
weights3 = np.array(weights[2])
weights4 = np.array(weights[3])
bias1 = np.array(weights[4])
bias2 = np.array(weights[5])
bias3 = np.array(weights[6])
bias4 = np.array(weights[7])
weightss = [weights1,weights2,weights3,weights4]
biases = [bias1,bias2,bias3,bias4]

# computational cost

In [ ]:
# computational cost of the material model
%%timeit
for i in range(num_steps-1):
    Tra[i+1,:], Dmg[i+1], Xid[i+1], Ene[i+1] = TSL(Gap[i+1,:], Dmg[i], Xid[i], prop)

2.17 ms ± 562 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# computational cost of the COMM_PINN
%%timeit
for k in range(len(Gap)-1):
    inp = [ x1_NN[k+1], x2_NN[k+1], x3_NN[k+1], x4_NN[k], x5_NN[k] ]
    x4_NN[k+1]   = feed_forward(inp , weightss, biases)

706 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
